In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install xlrd


DEPRECATION: beakerx-base 2.2.0 has a non-standard dependency specifier ipywidgets<8pandas,>=7.5.1. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of beakerx-base or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
import xlrd
print(f"xlrd version: {xlrd.__version__}")


xlrd version: 2.0.1


In [4]:
# Importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score, recall_score
# Removed shap import as per the user's request
import pyarrow

# Define the path to your Excel file
file_path = 'Annual change in the number of residences by type of activity and by municipality size class.xls'

# Load the Excel file
# Note: 'xlrd' engine supports only .xls files. Ensure your file is in .xls format.
excel_file = pd.ExcelFile(file_path, engine='xlrd')

# Retrieve the list of sheet names
sheet_names = excel_file.sheet_names
print(f"Available Sheets: {sheet_names}")

# Dictionary to store cleaned DataFrames keyed by sheet name
cleaned_data = {}

def combine_header_rows(df, header_start=0, header_end=9, join_separator=''):
    """
    Combines multiple header rows into a single header by concatenating the text.

    Parameters:
    - df: pandas DataFrame read without headers.
    - header_start: Starting row index for headers (inclusive).
    - header_end: Ending row index for headers (exclusive).
    - join_separator: Separator to use when joining header strings.

    Returns:
    - df_cleaned: DataFrame with combined headers and header rows removed.
    - combined_headers: List of combined header strings.
    """
    # Extract header rows
    header_data = df.iloc[header_start:header_end].fillna('')  # Replace NaN with empty strings
    
    # Combine header rows for each column by concatenating non-empty strings
    combined_headers = header_data.apply(
        lambda x: join_separator.join([str(item).strip() for item in x if str(item).strip() != '']),
        axis=0
    )
    
    # Assign combined headers to the DataFrame
    df.columns = combined_headers
    
    # Remove the header rows from the DataFrame
    df_cleaned = df.iloc[header_end:].reset_index(drop=True)
    
    return df_cleaned, combined_headers.tolist()

# Iterate over each sheet
for sheet in sheet_names:
    print(f"\nProcessing Sheet: {sheet}")
    
    # Read the sheet without headers
    df = pd.read_excel(file_path, sheet_name=sheet, header=None, engine='xlrd')
    
    # Combine header rows (rows 0 through 8) into a single header
    df_cleaned, combined_headers = combine_header_rows(df, header_start=0, header_end=9, join_separator='')
    
    # Display combined headers for verification
    print("Combined Headers:")
    for idx, header in enumerate(combined_headers):
        print(f"Column {idx}: '{header}'")
    
    # Remove columns where the combined header is empty
    # This handles any columns that were entirely empty in header rows
    non_empty_columns = [col for col in df_cleaned.columns if col.strip() != '']
    df_cleaned = df_cleaned[non_empty_columns]
    
    # Reset index after removing columns
    df_cleaned.reset_index(drop=True, inplace=True)
    
    # Drop any rows that are completely NaN
    df_cleaned.dropna(how='all', inplace=True)
    
    # Reset index after dropping rows
    df_cleaned.reset_index(drop=True, inplace=True)
    
    # Remove footer/metadata rows if known (e.g., last 3 rows)
    # Adjust the number based on your data
    # Here, assuming last 3 rows are metadata
    if df_cleaned.shape[0] > 3:
        df_cleaned = df_cleaned.iloc[:-3]  # Remove last 3 rows
    else:
        print(f"Warning: Sheet '{sheet}' has less than or equal to 3 data rows. Skipping footer removal.")
    
    # Replace remaining NaN values if necessary
    # For numerical data, fill with 0
    df_cleaned.fillna(0, inplace=True)
    
    # **Assign 'Year' based on sheet name instead of trying to access 'Year' column**
    # This avoids the KeyError since 'Year' might not be a separate column
    df_cleaned['Year'] = sheet
    
    # Display a preview of the cleaned DataFrame
    print("Cleaned DataFrame Preview:")
    display(df_cleaned.head())
    
    # Store the cleaned DataFrame in the dictionary
    cleaned_data[sheet] = df_cleaned

# Example Access to a Cleaned DataFrame
# Accessing df_2002
df_2002 = cleaned_data['2002']
print("\nDataFrame for 2002:")
display(df_2002)


Available Sheets: ['2009', '2008', '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000', '1999', '1998']

Processing Sheet: 2009
Combined Headers:
Column 0: ''
Column 1: 'Annual change in the number of residencesby type of activity and by municipality size class. 2009Municipalitysize classes'
Column 2: ''
Column 3: 'Increase as a result of ...New buildings'
Column 4: ''
Column 5: 'Conversion'
Column 6: ''
Column 7: 'Decrease as a result of ...Demolition, fire etc.'
Column 8: ''
Column 9: 'T 09.01.02Pure increase'
Cleaned DataFrame Preview:


,Annual change in the number of residencesby type of activity and by municipality size class. 2009Municipalitysize classes,Increase as a result of ...New buildings,Conversion,"Decrease as a result of ...Demolition, fire etc.",T 09.01.02Pure increase,Year
0,"more than 100,000 inhabitants",2350,220,649,1921,2009
1,"10,001 - 100,000 inhabitants",11339,441,1287,10493,2009
2,"5,001 - 10,000 inhabitants",7068,373,625,6816,2009
3,"2,001 - 5,000 inhabitants",11005,669,578,11096,2009
4,"2,000 inhabitants or fewer",7971,1000,320,8651,2009



Processing Sheet: 2008
Combined Headers:
Column 0: ''
Column 1: 'Annual change in the number of residencesby type of activity and by municipality size class. 2008Municipalitysize classes'
Column 2: ''
Column 3: 'Increase as a result of ...New buildings'
Column 4: ''
Column 5: 'Conversion'
Column 6: ''
Column 7: 'Decrease as a result of ...Demolition, fire etc.'
Column 8: ''
Column 9: 'T 09.01.02Pure increase'
Cleaned DataFrame Preview:


,Annual change in the number of residencesby type of activity and by municipality size class. 2008Municipalitysize classes,Increase as a result of ...New buildings,Conversion,"Decrease as a result of ...Demolition, fire etc.",T 09.01.02Pure increase,Year
0,"more than 100,000 inhabitants",2558,114,828,1844,2008
1,"10,001 - 100,000 inhabitants",12063,471,683,11851,2008
2,"5,001 - 10,000 inhabitants",8767,401,412,8756,2008
3,"2,001 - 5,000 inhabitants",11614,1040,410,12244,2008
4,"2,000 inhabitants or fewer",9189,1085,252,10022,2008



Processing Sheet: 2007
Combined Headers:
Column 0: ''
Column 1: 'Annual change in the number of residencesby type of activity and by municipality size class. 2007Municipalitysize classes'
Column 2: ''
Column 3: 'Increase as a result of ...New buildings'
Column 4: ''
Column 5: 'Conversion'
Column 6: ''
Column 7: 'Decrease as a result of ...Demolition, fire etc.'
Column 8: ''
Column 9: 'T 09.01.02Pure increase'
Cleaned DataFrame Preview:


...

,Annual change in the number of residencesby type of activity and by municipality size class. 1999Municipalitysize classes,Increase as a result of ...New buildings,Conversion,"Decrease as a result of ...Demolition, fire etc.",T 09.01.02Pure increase,Year
0,"more than 100,000 inhabitants",1416,102,341,1177,1999
1,"10,001 - 100,000 inhabitants",7639,229,367,7501,1999
2,"5,001 - 10,000 inhabitants",7009,269,148,7130,1999
3,"2,001 - 5,000 inhabitants",9027,735,243,9519,1999
4,"2,000 inhabitants or fewer",8017,1473,168,9322,1999



Processing Sheet: 1998
Combined Headers:
Column 0: 'Annual change in the number of residencesby type of activity and by municipality size class. 1998Municipalitysize classes'
Column 1: 'Increase as a result of ...New buildings'
Column 2: 'Conversion'
Column 3: 'Decrease as a result of ...Demolition, fire etc.'
Column 4: 'T 09.01.02Pure increase'
Cleaned DataFrame Preview:


,Annual change in the number of residencesby type of activity and by municipality size class. 1998Municipalitysize classes,Increase as a result of ...New buildings,Conversion,"Decrease as a result of ...Demolition, fire etc.",T 09.01.02Pure increase,Year
0,"more than 100,000 inhabitants",1818,78,370,1526,1998
1,"10,001 - 100,000 inhabitants",7887,319,429,7777,1998
2,"5,001 - 10,000 inhabitants",6539,337,159,6717,1998
3,"2,001 - 5,000 inhabitants",9215,644,241,9618,1998
4,"2,000 inhabitants or fewer",8275,1415,161,9529,1998



DataFrame for 2002:


,Annual change in the number of residencesby type of activity and by municipality size class. 2002Municipalitysize classes,Increase as a result of ...New buildings,Conversion 1),"Decrease as a result of ...Demolition, fire etc.",T 09.01.02Pure increase,Year
0,"more than 100,000 inhabitants",1527,-113,309,1105,2002
1,"10,001 - 100,000 inhabitants",7864,560,293,8131,2002
2,"5,001 - 10,000 inhabitants",6160,452,183,6429,2002
3,"2,001 - 5,000 inhabitants",7214,741,194,7761,2002
4,"2,000 inhabitants or fewer",5879,1129,111,6897,2002
5,Agglomerations of the five largest cities,10725,198,503,10420,2002
6,1) Negative values indicate a decrease caused ...,0,0,0,0,2002


In [5]:
# # Define the path to your Excel file
# file_path = 'Annual change in the number of residences by type of activity and by municipality size class.xls'

# # Load the Excel file
# excel_file = pd.ExcelFile(file_path, engine='xlrd')

# # Retrieve the list of sheet names
# sheet_names = excel_file.sheet_names
# print(f"Available Sheets: {sheet_names}")

# # Dictionary to store cleaned DataFrames keyed by sheet name
# cleaned_data = {}

# def combine_header_rows(df, header_start=0, header_end=9, join_separator=''):
#     # Combine header rows as before
#     header_data = df.iloc[header_start:header_end].fillna('')
#     combined_headers = header_data.apply(
#         lambda x: join_separator.join([str(item).strip() for item in x if str(item).strip() != '']),
#         axis=0
#     )
#     df.columns = combined_headers
    
#     # Remove header rows
#     df_cleaned = df.iloc[header_end:].reset_index(drop=True)
    
#     # Remove empty columns
#     non_empty_columns = [col for col in df_cleaned.columns if col.strip() != '']
#     df_cleaned = df_cleaned[non_empty_columns]
    
#     # Drop completely NaN rows
#     df_cleaned.dropna(how='all', inplace=True)
#     df_cleaned.reset_index(drop=True, inplace=True)
    
#     # **Optional:** Dynamically determine footer rows instead of removing last three
#     # For example, remove rows where 'Year' column is not equal to the sheet year
#     # Assuming 'Year' column exists and matches the sheet year
#     sheet_year = df_cleaned['Year'].iloc[0]
#     df_cleaned = df_cleaned[df_cleaned['Year'] == sheet_year]
    
#     # Replace remaining NaN values if necessary
#     df_cleaned.fillna(0, inplace=True)
    
#     return df_cleaned, combined_headers.tolist()



# # Iterate over each sheet
# for sheet in sheet_names:
#     print(f"\nProcessing Sheet: {sheet}")
    
#     # Read the sheet without headers
#     df = pd.read_excel(file_path, sheet_name=sheet, header=None, engine='xlrd')
    
#     # Combine header rows (rows 0 through 8) into a single header
#     df_cleaned, combined_headers = combine_header_rows(df, header_start=0, header_end=9, join_separator='')
    
#     # Display combined headers for verification
#     print("Combined Headers:")
#     for idx, header in enumerate(combined_headers):
#         print(f"Column {idx}: '{header}'")
    
#     # Remove columns where the combined header is empty
#     # This handles any columns that were entirely empty in header rows
#     non_empty_columns = [col for col in df_cleaned.columns if col.strip() != '']
#     df_cleaned = df_cleaned[non_empty_columns]
    
#     # Reset index after removing columns
#     df_cleaned.reset_index(drop=True, inplace=True)
    
#     # Drop any rows that are completely NaN
#     df_cleaned.dropna(how='all', inplace=True)
    
#     # Reset index after dropping rows
#     df_cleaned.reset_index(drop=True, inplace=True)
    
#     # Remove footer/metadata rows if known (e.g., last 3 rows)
#     # Adjust the number based on your data
#     # Here, assuming last 3 rows are metadata
#     df_cleaned = df_cleaned.iloc[:-3]  # Remove last 3 rows
    
#     # Replace remaining NaN values if necessary
#     # For numerical data, fill with 0
#     df_cleaned.fillna(0, inplace=True)
    
#     # Identify columns that should be numeric based on combined headers
#     # This requires knowledge of the header naming; adjust accordingly
#     # For example, columns containing 'increase' or 'decrease' are numeric
#     numeric_cols = [col for col in df_cleaned.columns if 
#                    'increase' in col.lower() or 'decrease' in col.lower()]
    
#     # Convert identified columns to numeric types
#     for col in numeric_cols:
#         # Remove any non-numeric characters like commas or spaces
#         df_cleaned[col] = df_cleaned[col].astype(str).str.replace(',', '').str.replace(' ', '')
#         # Convert to numeric, coercing errors to NaN
#         df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors='coerce')
#         # Fill NaN values with 0 or appropriate value
#         df_cleaned[col].fillna(0, inplace=True)
    
#     # Add a 'Year' column based on the sheet name
#     df_cleaned['Year'] = sheet
    
#     # Store the cleaned DataFrame in the dictionary
#     cleaned_data[sheet] = df_cleaned
    
#     # Display a preview of the cleaned DataFrame
#     print("Cleaned DataFrame Preview:")
#     display(df_cleaned.head())


In [6]:
df_1998 = cleaned_data['1998']
df_1999 = cleaned_data['1999']
df_2000 = cleaned_data['2000']
df_2001 = cleaned_data['2001']
df_2002 = cleaned_data['2002']
df_2003 = cleaned_data['2003']
df_2004 = cleaned_data['2004']
df_2005 = cleaned_data['2005']
df_2006 = cleaned_data['2006']
df_2007 = cleaned_data['2007']
df_2008 = cleaned_data['2008']
df_2009 = cleaned_data['2009']

In [7]:
df_2003

,Annual change in the number of residencesby type of activity and by municipality size class. 2003Municipalitysize classes,Increase as a result of ...New buildings,Conversion 1),"Decrease as a result of ...Demolition, fire etc.",T 09.01.02Pure increase,Year
0,"more than 100,000 inhabitants",1934,-50,221,1663,2003
1,"10,001 - 100,000 inhabitants",9350,425,395,9380,2003
2,"5,001 - 10,000 inhabitants",6858,417,202,7073,2003
3,"2,001 - 5,000 inhabitants",7617,980,294,8303,2003
4,"2,000 inhabitants or fewer",6337,1165,216,7286,2003
5,Agglomerations of the five largest cities,11558,354,428,11484,2003
6,1) Negative values indicate a decrease caused ...,0,0,0,0,2003


In [8]:
df_2008

,Annual change in the number of residencesby type of activity and by municipality size class. 2008Municipalitysize classes,Increase as a result of ...New buildings,Conversion,"Decrease as a result of ...Demolition, fire etc.",T 09.01.02Pure increase,Year
0,"more than 100,000 inhabitants",2558,114,828,1844,2008
1,"10,001 - 100,000 inhabitants",12063,471,683,11851,2008
2,"5,001 - 10,000 inhabitants",8767,401,412,8756,2008
3,"2,001 - 5,000 inhabitants",11614,1040,410,12244,2008
4,"2,000 inhabitants or fewer",9189,1085,252,10022,2008
5,Agglomerations of the five largest cities,14003,348,1446,12905,2008


In [9]:
df2009 = df_2009.iloc[:-1].copy()
df2008 = df_2009.iloc[:-1].copy()
df2007 = df_2009.iloc[:-1].copy()
df2006 = df_2009.iloc[:-1].copy()
df2005 = df_2009.iloc[:-1].copy()
df2004 = df_2009.iloc[:-1].copy()
df2003 = df_2009.iloc[:-1].copy()
df2002 = df_2009.iloc[:-1].copy()
df2001 = df_2009.iloc[:-1].copy()
df2000 = df_2009.iloc[:-1].copy()
df1999 = df_1999.iloc[:-1].copy()
df1998 = df_1998.iloc[:-1].copy()


In [10]:
df2009

,Annual change in the number of residencesby type of activity and by municipality size class. 2009Municipalitysize classes,Increase as a result of ...New buildings,Conversion,"Decrease as a result of ...Demolition, fire etc.",T 09.01.02Pure increase,Year
0,"more than 100,000 inhabitants",2350,220,649,1921,2009
1,"10,001 - 100,000 inhabitants",11339,441,1287,10493,2009
2,"5,001 - 10,000 inhabitants",7068,373,625,6816,2009
3,"2,001 - 5,000 inhabitants",11005,669,578,11096,2009
4,"2,000 inhabitants or fewer",7971,1000,320,8651,2009


In [11]:
df1998

,Annual change in the number of residencesby type of activity and by municipality size class. 1998Municipalitysize classes,Increase as a result of ...New buildings,Conversion,"Decrease as a result of ...Demolition, fire etc.",T 09.01.02Pure increase,Year
0,"more than 100,000 inhabitants",1818,78,370,1526,1998
1,"10,001 - 100,000 inhabitants",7887,319,429,7777,1998
2,"5,001 - 10,000 inhabitants",6539,337,159,6717,1998
3,"2,001 - 5,000 inhabitants",9215,644,241,9618,1998
4,"2,000 inhabitants or fewer",8275,1415,161,9529,1998


In [12]:
df2002

,Annual change in the number of residencesby type of activity and by municipality size class. 2009Municipalitysize classes,Increase as a result of ...New buildings,Conversion,"Decrease as a result of ...Demolition, fire etc.",T 09.01.02Pure increase,Year
0,"more than 100,000 inhabitants",2350,220,649,1921,2009
1,"10,001 - 100,000 inhabitants",11339,441,1287,10493,2009
2,"5,001 - 10,000 inhabitants",7068,373,625,6816,2009
3,"2,001 - 5,000 inhabitants",11005,669,578,11096,2009
4,"2,000 inhabitants or fewer",7971,1000,320,8651,2009


In [13]:
Total = df2002['T 09.01.02Pure increase'].sum()
Total

38977

In [14]:
df2009

,Annual change in the number of residencesby type of activity and by municipality size class. 2009Municipalitysize classes,Increase as a result of ...New buildings,Conversion,"Decrease as a result of ...Demolition, fire etc.",T 09.01.02Pure increase,Year
0,"more than 100,000 inhabitants",2350,220,649,1921,2009
1,"10,001 - 100,000 inhabitants",11339,441,1287,10493,2009
2,"5,001 - 10,000 inhabitants",7068,373,625,6816,2009
3,"2,001 - 5,000 inhabitants",11005,669,578,11096,2009
4,"2,000 inhabitants or fewer",7971,1000,320,8651,2009


In [15]:
df2009.columns = df2009.columns.str.strip()

# Step 2: Identify numeric columns by position (all except first and last)
numeric_cols = df2009.columns[1:-1].tolist()

# Convert numeric columns to numeric types, handling non-numeric characters
def convert_to_numeric(df, columns):
    for col in columns:
        # Remove commas and any other non-numeric characters if necessary
        df[col] = df[col].astype(str).str.replace(',', '').str.strip()
        # Convert to numeric, coercing errors to NaN
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
    return df

# Apply the conversion function
df2009 = convert_to_numeric(df2009, numeric_cols)

# Convert 'Year' to numeric if it's not already
df2009['Year'] = pd.to_numeric(df2009['Year'], errors='coerce').fillna(0).astype(int)

# Step 3: Calculate sums for numeric columns
sum_values = df2009[numeric_cols].sum()

# Step 4: Create a dictionary for the Total row
total_dict = {
    df2009.columns[0]: 'Total',  # Label the first column as 'Total'
    'Increase as a result of ...New buildings': sum_values['Increase as a result of ...New buildings'],
    'Conversion': sum_values['Conversion'],
    'Decrease as a result of ...Demolition, fire etc.': sum_values['Decrease as a result of ...Demolition, fire etc.'],
    'T 09.01.02Pure increase': sum_values['T 09.01.02Pure increase'],
    'Year': df2009['Year'].iloc[0]  # Assuming all entries have the same year
}

# Step 5: Create a DataFrame for the Total row
total_row_df = pd.DataFrame([total_dict])

# Ensure the Total row has all columns in the correct order
total_row_df = total_row_df[df2009.columns]

# Step 6: Append the Total row to the original DataFrame
df2009_with_total = pd.concat([df2009, total_row_df], ignore_index=True)

# Display the updated DataFrame
print("\nDataFrame for 2009 with Total Row:")
display(df2009_with_total)


DataFrame for 2009 with Total Row:


,Annual change in the number of residencesby type of activity and by municipality size class. 2009Municipalitysize classes,Increase as a result of ...New buildings,Conversion,"Decrease as a result of ...Demolition, fire etc.",T 09.01.02Pure increase,Year
0,"more than 100,000 inhabitants",2350,220,649,1921,2009
1,"10,001 - 100,000 inhabitants",11339,441,1287,10493,2009
2,"5,001 - 10,000 inhabitants",7068,373,625,6816,2009
3,"2,001 - 5,000 inhabitants",11005,669,578,11096,2009
4,"2,000 inhabitants or fewer",7971,1000,320,8651,2009
5,Total,39733,2703,3459,38977,2009


In [16]:
df1998

,Annual change in the number of residencesby type of activity and by municipality size class. 1998Municipalitysize classes,Increase as a result of ...New buildings,Conversion,"Decrease as a result of ...Demolition, fire etc.",T 09.01.02Pure increase,Year
0,"more than 100,000 inhabitants",1818,78,370,1526,1998
1,"10,001 - 100,000 inhabitants",7887,319,429,7777,1998
2,"5,001 - 10,000 inhabitants",6539,337,159,6717,1998
3,"2,001 - 5,000 inhabitants",9215,644,241,9618,1998
4,"2,000 inhabitants or fewer",8275,1415,161,9529,1998


In [21]:
import pandas as pd
import numpy as np

# Step 1: Organize Your DataFrames
# List of years corresponding to your DataFrames
years = range(1998, 2010)  # 1998 through 2009

# Initialize an empty dictionary to store the DataFrames
dfs = {}

# Populate the dictionary with existing DataFrames
for year in years:
    df_name = f'df{year}'  # e.g., 'df1998', 'df1999', ..., 'df2009'
    
    if df_name in globals():
        full_year = str(year)  # e.g., '1998', '1999', ..., '2009'
        dfs[full_year] = globals()[df_name]
    else:
        print(f"Warning: {df_name} does not exist in the global namespace.")

# Step 2: Define the Function to Add a "Total" Row
def add_total_row(df, year_column='Year'):
    """
    Adds a 'Total' row to the DataFrame with sums of numeric columns.

    Parameters:
    - df (pd.DataFrame): The DataFrame to process.
    - year_column (str): The name of the year column.

    Returns:
    - pd.DataFrame: The updated DataFrame with the 'Total' row appended.
    """
    try:
        # Step 1: Clean column names by stripping leading/trailing spaces
        df.columns = df.columns.str.strip()

        # Step 2: Identify numeric columns dynamically
        # Assume that all columns except the first and last are numeric
        numeric_cols = df.columns[1:-1].tolist()

        # Step 3: Convert identified columns to numeric types
        for col in numeric_cols:
            if col in df.columns:
                # Remove commas and any other non-numeric characters if present
                df[col] = df[col].astype(str).str.replace(',', '').str.strip()
                # Convert to numeric, coercing errors to NaN
                df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
            else:
                # If the column does not exist, assign 0 and print a warning
                print(f"Warning: Column '{col}' not found in DataFrame.")
                df[col] = 0

        # Also, ensure the 'Year' column is numeric
        if year_column in df.columns:
            df[year_column] = pd.to_numeric(df[year_column], errors='coerce').fillna(0).astype(int)
        else:
            print(f"Warning: Year column '{year_column}' not found in DataFrame.")
            df[year_column] = 0

        # Step 4: Calculate sums for numeric columns
        sum_values = df[numeric_cols].sum()

        # Step 5: Create a dictionary for the Total row
        first_column = df.columns[0]
        total_dict = {
            first_column: 'Total'
        }
        for col in numeric_cols:
            if col in sum_values:
                total_dict[col] = sum_values[col]
            else:
                total_dict[col] = 0  # Assign 0 if the column is missing
        if year_column in df.columns:
            total_dict[year_column] = df[year_column].iloc[0]  # Assuming all entries have the same year
        else:
            total_dict[year_column] = 0  # Assign 0 or another default value

        # Step 6: Create a DataFrame for the Total row
        total_row_df = pd.DataFrame([total_dict])

        # Step 7: Ensure the Total row has all columns in the correct order
        total_row_df = total_row_df[df.columns]

        # Step 8: Append the Total row to the original DataFrame
        df_with_total = pd.concat([df, total_row_df], ignore_index=True)

        return df_with_total

    except Exception as e:
        print(f"An error occurred while processing the DataFrame: {e}")
        return df

# Step 3: Apply the Function to All DataFrames
# Initialize a new dictionary to store updated DataFrames
dfs_with_total = {}

for year, df in dfs.items():
    print(f"Processing DataFrame for Year: {year}")
    updated_df = add_total_row(
        df,
        year_column='Year'
    )
    dfs_with_total[year] = updated_df

# Step 4: Access and Verify the Updated DataFrames
# Example: Display the updated DataFrame for 2009
if '2009' in dfs_with_total:
    print("\nDataFrame for 2009 with Total Row:")
    display(dfs_with_total['2009'])
else:
    print("DataFrame for 2009 is not available in dfs_with_total.")

# Optionally, display all updated DataFrames
for year, df in dfs_with_total.items():
    print(f"\nDataFrame for {year} with Total Row:")
    display(df)

# Step 5: Overwrite the Original DataFrames with Updated Ones (Optional)
for year, df in dfs_with_total.items():
    df_name = f'df{year}'
    globals()[df_name] = df
    print(f"Updated {df_name} with Total row.")


Processing DataFrame for Year: 1998
Processing DataFrame for Year: 1999
Processing DataFrame for Year: 2000
Processing DataFrame for Year: 2001
Processing DataFrame for Year: 2002
Processing DataFrame for Year: 2003
Processing DataFrame for Year: 2004
Processing DataFrame for Year: 2005
Processing DataFrame for Year: 2006
Processing DataFrame for Year: 2007
Processing DataFrame for Year: 2008
Processing DataFrame for Year: 2009

DataFrame for 2009 with Total Row:


,Annual change in the number of residencesby type of activity and by municipality size class. 2009Municipalitysize classes,Increase as a result of ...New buildings,Conversion,"Decrease as a result of ...Demolition, fire etc.",T 09.01.02Pure increase,Year
0,"more than 100,000 inhabitants",2350,220,649,1921,2009
1,"10,001 - 100,000 inhabitants",11339,441,1287,10493,2009
2,"5,001 - 10,000 inhabitants",7068,373,625,6816,2009
3,"2,001 - 5,000 inhabitants",11005,669,578,11096,2009
4,"2,000 inhabitants or fewer",7971,1000,320,8651,2009
5,Total,39733,2703,3459,38977,2009



DataFrame for 1998 with Total Row:


,Annual change in the number of residencesby type of activity and by municipality size class. 1998Municipalitysize classes,Increase as a result of ...New buildings,Conversion,"Decrease as a result of ...Demolition, fire etc.",T 09.01.02Pure increase,Year
0,"more than 100,000 inhabitants",1818,78,370,1526,1998
1,"10,001 - 100,000 inhabitants",7887,319,429,7777,1998
2,"5,001 - 10,000 inhabitants",6539,337,159,6717,1998
3,"2,001 - 5,000 inhabitants",9215,644,241,9618,1998
4,"2,000 inhabitants or fewer",8275,1415,161,9529,1998
5,Total,33734,2793,1360,35167,1998



DataFrame for 1999 with Total Row:


...


DataFrame for 2008 with Total Row:


,Annual change in the number of residencesby type of activity and by municipality size class. 2009Municipalitysize classes,Increase as a result of ...New buildings,Conversion,"Decrease as a result of ...Demolition, fire etc.",T 09.01.02Pure increase,Year
0,"more than 100,000 inhabitants",2350,220,649,1921,2009
1,"10,001 - 100,000 inhabitants",11339,441,1287,10493,2009
2,"5,001 - 10,000 inhabitants",7068,373,625,6816,2009
3,"2,001 - 5,000 inhabitants",11005,669,578,11096,2009
4,"2,000 inhabitants or fewer",7971,1000,320,8651,2009
5,Total,39733,2703,3459,38977,2009



DataFrame for 2009 with Total Row:


,Annual change in the number of residencesby type of activity and by municipality size class. 2009Municipalitysize classes,Increase as a result of ...New buildings,Conversion,"Decrease as a result of ...Demolition, fire etc.",T 09.01.02Pure increase,Year
0,"more than 100,000 inhabitants",2350,220,649,1921,2009
1,"10,001 - 100,000 inhabitants",11339,441,1287,10493,2009
2,"5,001 - 10,000 inhabitants",7068,373,625,6816,2009
3,"2,001 - 5,000 inhabitants",11005,669,578,11096,2009
4,"2,000 inhabitants or fewer",7971,1000,320,8651,2009
5,Total,39733,2703,3459,38977,2009


Updated df1998 with Total row.
Updated df1999 with Total row.
Updated df2000 with Total row.
Updated df2001 with Total row.
Updated df2002 with Total row.
Updated df2003 with Total row.
Updated df2004 with Total row.
Updated df2005 with Total row.
Updated df2006 with Total row.
Updated df2007 with Total row.
Updated df2008 with Total row.
Updated df2009 with Total row.


In [24]:
df2007

,Annual change in the number of residencesby type of activity and by municipality size class. 2009Municipalitysize classes,Increase as a result of ...New buildings,Conversion,"Decrease as a result of ...Demolition, fire etc.",T 09.01.02Pure increase,Year
0,"more than 100,000 inhabitants",2350,220,649,1921,2009
1,"10,001 - 100,000 inhabitants",11339,441,1287,10493,2009
2,"5,001 - 10,000 inhabitants",7068,373,625,6816,2009
3,"2,001 - 5,000 inhabitants",11005,669,578,11096,2009
4,"2,000 inhabitants or fewer",7971,1000,320,8651,2009
5,Total,39733,2703,3459,38977,2009
